In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,37502
2,Huelva,Confirmados PDIA 14 días,2691
3,Huelva,Tasa PDIA 14 días,"524,3876298302707"
4,Huelva,Confirmados PDIA 7 días,1364
5,Huelva,Tasa PDIA 7 dias,"265,79885807821967"
6,Huelva,Total Confirmados,37709
7,Huelva,Curados,33369
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  37502.0


/tmp/ipykernel_102929/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10480.0


/tmp/ipykernel_102929/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_102929/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_102929/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_102929/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 338 personas en los últimos 7 días 

Un positivo PCR cada 156 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,37502.0,1364.0,2691.0,513170.0,265.798858,524.387630,221.0
Huelva-Costa,22048.0,890.0,1829.0,289548.0,307.375634,631.674196,153.0
Huelva (capital),10480.0,426.0,920.0,143837.0,296.168580,639.612895,81.0
Condado-Campiña,11795.0,310.0,582.0,156231.0,198.424128,372.525299,49.0
Sierra de Huelva-Andévalo Central,3286.0,164.0,278.0,67391.0,243.355938,412.517992,27.0
Aljaraque,1389.0,78.0,155.0,21474.0,363.229953,721.803111,19.0
Punta Umbría,1008.0,70.0,123.0,15355.0,455.877564,801.042006,14.0
Gibraleón,752.0,42.0,63.0,12737.0,329.747978,494.621967,14.0
Lepe,2457.0,83.0,168.0,27880.0,297.704448,602.582496,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,137.0,19.0,19.0,1310.0,1450.381679,1450.381679,0.0
Cortelazor,17.0,2.0,4.0,299.0,668.896321,1337.792642,0.0
Beas,284.0,25.0,47.0,4341.0,575.904170,1082.699839,3.0
Santa Ana la Real,16.0,4.0,5.0,475.0,842.105263,1052.631579,0.0
Isla Cristina,2554.0,82.0,176.0,21393.0,383.302950,822.699014,8.0
Palos de la Frontera,860.0,47.0,95.0,11742.0,400.272526,809.061489,4.0
Punta Umbría,1008.0,70.0,123.0,15355.0,455.877564,801.042006,14.0
Nerva,221.0,18.0,38.0,5169.0,348.229832,735.151867,2.0
Aljaraque,1389.0,78.0,155.0,21474.0,363.229953,721.803111,19.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Higuera de la Sierra,38.0,1.0,4.0,1291.0,77.459334,309.837335,0.0,0.250000
Manzanilla,115.0,1.0,4.0,2118.0,47.214353,188.857413,0.0,0.250000
San Bartolomé de la Torre,320.0,2.0,7.0,3761.0,53.177346,186.120713,0.0,0.285714
Niebla,316.0,5.0,17.0,4116.0,121.477162,413.022352,1.0,0.294118
Alosno,210.0,1.0,3.0,3933.0,25.425884,76.277651,0.0,0.333333
Santa Bárbara de Casa,38.0,1.0,3.0,1043.0,95.877277,287.631831,0.0,0.333333
Jabugo,76.0,3.0,8.0,2260.0,132.743363,353.982301,1.0,0.375000
Almonaster la Real,75.0,2.0,5.0,1788.0,111.856823,279.642058,0.0,0.400000
Campillo (El),82.0,3.0,7.0,2024.0,148.221344,345.849802,0.0,0.428571
